In [ ]:
import requests
from bs4 import BeautifulSoup
import wget
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

In [ ]:
path_to_directory = 'PATH_TO_YOUR_FOLDER'

In [ ]:
def download_podcasts(soup, title):
  i = 0
  time_all = [] #Dates of realeases of the podcast episodes
  ID_all = [] #ID of the podcast episodes
  description_all = [] #Description of each podcast episode
  length_all = [] #Duration of each podcast episode
  link_all = [] #Link to each podcast episode
  name_all = [] #Names of the podcast episodes
  for podcast in soup.find_all('a', {'role': 'listitem'}):
    time = podcast.find('div', {'class': 'OTz6ee'}).text
    #Downloading all episodes released in December 2022
    if (time.startswith('Dec')) and (time.endswith('2022')):
      i += 1
      time_all.append(time) #Date of release of the episode

      link = podcast.find('div', {'jsname': 'fvi9Ef'})['jsdata'].split(';')[1]
      link_all.append(link) #Link for the episode

      name = podcast.find('div', {'class': 'e3ZUqe'}).text
      name_all.append(name) #Name of the episode

      print(i, ':', time)
      filename = wget.download(link, out=path_to_directory+title) #Downloading the episode audio file
      os.rename(filename, path_to_directory+title+'/audio'+str(i)+'.mp3')
      ID_all.append('audio'+str(i)) #ID for the episode

      #Looking to see if the description of the episode is present
      try:
        description = podcast.find('div', {'class': 'LrApYe'}).text
      except:
        description = 'None'

      if description is None:
        description = 'None'
      description_all.append(description)

      length = podcast.find('span', {'class': 'gUJ0Wc'}).text
      length_all.append(length) #Duration of the episode

  #Saving all the information relating to the podcast episode as a pandas dataframe
  df = pd.DataFrame(list(zip([title]*len(ID_all), ID_all, name_all,
                             time_all,
                             description_all,
                             length_all,
                             link_all)),
                    columns = ['Show', 'ID', 'Episode', 'Time',
                               'Description', 'Length', 'Link'])
  return df

In [ ]:
#URLs of the podcasts: 'The Stack Overflow Podcast' and 'Myths and Legends'
URLs = ['LINKS_TO_PODCASTS']

info_df_list = []

for url in URLs:
  #Using BeautifulSoup to scrape and parse HTML content from the web pages of the two podcasts using lxml parser
  soup = BeautifulSoup(requests.get(url).text, 'lxml')
  title = soup.find('div', {'class': 'ZfMIwb'}).text #To retrieve the title of the podcast
  os.mkdir(path_to_directory+title)
  df = download_podcasts(soup, title) #Calling the previously defined function to download the relevant episodes and return their data
  info_df_list.append(df)

info_all_podcasts = pd.concat(info_df_list)

1 : Dec 20, 2022
2 : Dec 16, 2022
3 : Dec 13, 2022
4 : Dec 12, 2022
5 : Dec 9, 2022
6 : Dec 6, 2022
7 : Dec 2, 2022
1 : Dec 27, 2022
2 : Dec 20, 2022
3 : Dec 13, 2022
4 : Dec 6, 2022


In [ ]:
info_all_podcasts

,Show,ID,Episode,Time,Description,Length,Link
0,The Stack Overflow Podcast,audio1,Let's talk about our favorite terminal tools,"Dec 20, 2022",You can learn more about Anthony here.\n\nHis ...,25 min,https://chrt.fm/track/G8F1AF/afp-71129-injecte...
1,The Stack Overflow Podcast,audio2,An honest end-of-year rundown,"Dec 16, 2022",Ben asks Matt to explain Mastodon to him like ...,17 min,https://chrt.fm/track/G8F1AF/afp-71129-injecte...
2,The Stack Overflow Podcast,audio3,Talking about drag and drop tech stacks with B...,"Dec 13, 2022",Steve was working as an engineering manager at...,24 min,https://chrt.fm/track/G8F1AF/afp-71129-injecte...
3,The Stack Overflow Podcast,audio4,The next step in ecommerce? Replatform with AP...,"Dec 12, 2022",SPONSORED BY COMMERCE LAYER\n\nAround the worl...,26 min,https://chrt.fm/track/G8F1AF/afp-71129-injecte...
4,The Stack Overflow Podcast,audio5,Ready to optimize your JavaScript with Rust?,"Dec 9, 2022",Webpack has been king for several years. Verce...,23 min,https://chrt.fm/track/G8F1AF/afp-71129-injecte...
5,The Stack Overflow Podcast,audio6,The tech to build in a crypto winter,"Dec 6, 2022","You can learn more about Andrew, from building...",21 min,https://chrt.fm/track/G8F1AF/afp-71129-injecte...
6,The Stack Overflow Podcast,audio7,Taking stock of the crypto crash and tech turb...,"Dec 2, 2022",Data show's Silicon Valley's share of new star...,20 min,https://chrt.fm/track/G8F1AF/afp-71129-injecte...
0,Myths and Legends,audio1,301-Japanese Folklore: After Sundown,"Dec 27, 2022",The story of a family and the monsters that hu...,19 min,https://pscrb.fm/rss/p/pdst.fm/e/pfx.vpixl.com...
1,Myths and Legends,audio2,300-Norse Christmas: A Slaying Song Tonight,"Dec 20, 2022",An abbot spends Christmas surrounded by outlaw...,44 min,https://pscrb.fm/rss/p/pdst.fm/e/pfx.vpixl.com...
2,Myths and Legends,audio3,299-Korean Folklore: Long Con,"Dec 13, 2022",The spies are coming. Governor Sim has gone to...,43 min,https://pscrb.fm/rss/p/pdst.fm/e/pfx.vpixl.com...


In [ ]:
#Saving the dataframe as a csv file for future use
info_all_podcasts.to_csv(path_to_directory+'podcast.csv')